In [81]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import scipy as sp
import sensuous.preprocessing.csv as prep
from sensuous.model.knn import strip_artists
from sklearn.preprocessing import MinMaxScaler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
prep.preprocess_csv_data()

Csv data preprocessing done, saved the cleaned data to file "../data/all-songs.csv"


In [35]:
df = pd.read_csv('../data/all-songs.csv')

In [38]:
df = df.drop(columns='Unnamed: 0')

In [78]:
n_neighbors = 10

In [82]:
df_numeric = df.drop(columns=['artists', 'song_title'])
columns = df_numeric.columns.values.tolist()
df_scaler = MinMaxScaler()
df[columns] = df_scaler.fit_transform(df[columns])

# Seed song selection
song_title = 'Do It'
artist = 'Mike Stud'

criterion = (df['artists'].map(lambda x: artist in x)) & (
    df['song_title'] == song_title)
seed_song = df[criterion]

if seed_song.shape[0] == 0:
    print("No such a song in the dataset. Returning the user's entry.")
#     return f'{song} by {artist}'

# Feature "preprocessing"
seed_song_features = seed_song.select_dtypes(exclude='object')
X = df.select_dtypes(exclude='object')

# List of closest neighbors to the seed song
tree = sp.spatial.KDTree(X.to_numpy())
distances, indeces = tree.query([seed_song_features], k=n_neighbors)
indeces = indeces.flatten()
neighbors_df = df.loc[list(indeces)]
songs = neighbors_df['song_title'].tolist()
artists = [
    strip_artists(artist) for artist in neighbors_df['artists'].tolist()
]
playlist = {'playlist': list(zip(songs, artists))}

In [83]:
playlist

{'playlist': [('Do It', 'Mike Stud'),
  ('Chasing Tail', 'Win and Woo'),
  ('In the Club', 'Big Chris'),
  ('On A Mission', 'Summerella'),
  ('Punta Cana', 'Creatives Of'),
  ('No More', 'J9ueve'),
  ('Bumblebee', 'Smokepurpp'),
  ('Pataas Pilla', 'Sricharan Pakala, Anirudh Ravichander'),
  ('Lies About You (feat. Lil Durk)', 'Gunna'),
  ('Homework (feat. THEMXXNLIGHT)', 'Wiz Khalifa')]}